# Spam Email Detection
## Author: Hexing Ren
### Click [here](http://www.hexingren.com/practical-data-science) to go back.

# Introduction
In this project, we will classify emails as either spam or not spam using support vector machines. The full dataset consists 80k labeled emails. The labels are 1 if they are ham (not spam), and -1 if they are spam. The lines of the emails have already been slightly processed, such that different words are space delimited, however little other processing has occurred. 

## Preliminary notes
1. Scikit-learn is allowed to use. 
2. For this project, each proceeding part depends on the previous since we are building up a moderately sized data science pipeline. Verify your previous parts before proceeding onto the next. 
3. Similar the linear regression notebook of the previous project, you will need to use the tfidf function from the natural language processing notebook.
4. As we move into more advanced algorithms and techniques, there will be more introductions of randomness. This means that some of the example outputs in the notebook contain some randomness, and will probably not match the results exactly. Verify the code by checking the properties/invariants or feeding in static inputs for which we can calculate the output. 
5. When writing pickle files to be read into Autolab, **write files with the binary flag**

In [ ]:
import numpy as np
import scipy.sparse as sp
from collections import Counter
import scipy.optimize
import cPickle as pickle

In [ ]:
# AUTOLAB_IGNORE_START
with open("X1.txt") as f:
    emails = f.readlines()
labels = np.loadtxt("y1.txt")
# AUTOLAB_IGNORE_STOP

In [ ]:
# AUTOLAB_IGNORE_START
from natural_language_processing import tfidf
features, all_words = tfidf(emails)
# AUTOLAB_IGNORE_STOP

## SVM classification
Recall the support vector machine (SVM) from slide 17 of linear classification. Since it is such a straightforward algorithm, we will implement it below.

### Specifications
1. If you do not use matrix operations, your code will be **very slow**. Every function in here can be implemented in 1 or 2 lines using matrix equations, and the only for loop you need is the training loop for gradient descent. **If your code is slow here, it will be extremely slow in the next section when doing parameter search**.
2. You should train your SVM using gradient descent as described in the slides. Your objective value should also mimic that of the slides. 
3. Since this is a convex function, your gradient steps should always decrease your objective. A simple check when writing these optimization procedures is to print your objectives and verify that this is the case (or plot them with matplotlib).
4. You can also use scipy.optimize.check_grad to numerically verify the correctness of your gradients. 
5. For the unlikely boundary case where your hypothesis outputs 0, we will treat that as a positive prediction. 
6. Be careful of numpy.matrix objects which are constrained to always have dimension 2 (scipy operations will sometimes return this instead of an ndarray). 

In [ ]:
class SVM:
    def __init__(self, X, y, reg):
        """ Initialize the SVM attributes and initialize the weights vector to the zero vector. 
            Attributes: 
                X (array_like) : training data intputs
                y (vector) : 1D numpy array of training data outputs
                reg (float) : regularizer parameter
                theta : 1D numpy array of weights
        """
        self.X = X
        self.y = y
        self.reg = reg
        self.theta = np.zeros(X.shape[1])
        
        self.Xy = sp.diags(y).dot(X)
    
    def objective(self, X, y):
        """ Calculate the objective value of the SVM. When given the training data (self.X, self.y), this is the 
            actual objective being optimized. 
            Args:
                X (array_like) : array of examples, where each row is an example
                y (array_like) : array of outputs for the training examples
            Output:
                (float) : objective value of the SVM when calculated on X,y
        """
        Xy = sp.diags(y).dot(X)
        dist = (-1) * Xy.dot(self.theta) + 1
        obj = dist[dist > 0].sum() + (self.reg / 2) * (np.linalg.norm(self.theta)**2)
        return obj

    
    def gradient(self):
        """ Calculate the gradient of the objective value on the training examples. 
            Output:
                (vector) : 1D numpy array containing the gradient
        """
        grad = (-1) * ((self.Xy).T.dot((self.Xy).dot(self.theta) <= 1)) + self.reg * self.theta
        return grad
    
    def train(self, niters=100, learning_rate=1, verbose=False):
        """ Train the support vector machine with the given parameters. 
            Args: 
                niters (int) : the number of iterations of gradient descent to run
                learning_rate (float) : the learning rate (or step size) to use when training
                verbose (bool) : an optional parameter that you can use to print useful information (like objective value)
        """
        for iter in range(niters):
            self.theta = self.theta -  learning_rate * self.gradient()
            if verbose:
                print self.objective(self.X, self.y)
    
    def predict(self, X):
        """ Predict the class of each label in X. 
            Args: 
                X (array_like) : array of examples, where each row is an example
            Output:
                (vector) : 1D numpy array containing predicted labels
        """
        y_pre = np.array(X.dot(self.theta))
        y_pre[y_pre < 0] = -1
        y_pre[y_pre >= 0] = +1
        return y_pre

Some useful tricks for debugging: 
1. Use very simple inputs (i.e. small vectors of ones) and compare the output of each function with a hand calculation. 
2. One way to guarantee your gradient is correct is to verify it numerically using a derivative approximation. You can read more about numerical differentiation methods here (https://en.wikipedia.org/wiki/Finite_difference) but for your purposes, you can use scipy.optimize.check_grad to do the numerical checking for you. 

In [ ]:
# AUTOLAB_IGNORE_START
# Verify the correctness of your code on small examples
y0 = np.random.randint(0,2,5)*2-1
X0 = np.random.random((5,10))
t0 = np.random.random(10)
svm0 = SVM(X0,y0, 1e-4)
svm0.theta = t0


# def obj(theta):
#     pass

# def grad(theta):
#     pass

# scipy.optimize.check_grad(obj, grad, t0)

svm0.train(niters=100, learning_rate=1, verbose=True)
# AUTOLAB_IGNORE_STOP

On the above small example, our solution gets a gradient error on the order of 1e-08 from scipy.optimize.check_grad. Your objective values should be monotonically decreasing. 

Once that works, try training your SVM on the tfidf features.

In [ ]:
# AUTOLAB_IGNORE_START
svm = SVM(features, labels, reg = 1e-4)
svm.train(niters=100, learning_rate=1, verbose=False)
print (svm.predict(features) == labels).sum()
print len(labels)
print svm.predict(features)
# AUTOLAB_IGNORE_STOP

Our implementation gets the following results:
* For 100 iterations, regularization 1e-4, and learning rate 1.0, our solution is able to achieve perfect training classification accuracy (100% accuracy on the training data)
* Training for 100 iterations takes about 2.13 seconds (measured using %timeit). 

## Model Selection: Cross validation and Parameter Grid Search
As you may have noticed, there are parameters in the SVM learning algorithm that we chose somewhat arbitrarily: the regularization parameter and the learning rate (also technically the number of iterations for the learning algorithm, but you'll only consider the first two for simplicity). 

We were also able to achieve perfect training accuracy with these random parameters. This should make you suspicious: we have an enormous amount of features so it would be extremely easy to overfit to the data, so our model may not generalize well. 

You will now evaluate and select parameters using cross validation and grid search.

In [ ]:
import math

class ModelSelector:
    """ A class that performs model selection. 
        Attributes:
            blocks (list) : list of lists of indices of each block used for k-fold cross validation, e.g. blocks[i] 
            gives the indices of the examples in the ith block 
            test_block (list) : list of indices of the test block that used only for reporting results
            
    """
    def __init__(self, X, y, P, k, niters):
        """ Initialize the model selection with data and split into train/valid/test sets. Split the permutation into blocks 
            and save the block indices as an attribute to the model. 
            Args:
                X (array_like) : array of features for the datapoints
                y (vector) : 1D numpy array containing the output labels for the datapoints
                P (vector) : 1D numpy array containing a random permutation of the datapoints
                k (int) : number of folds
                niters (int) : number of iterations to train for
        """
        self.X = X
        self.y = y
        self.k = k
        self.niters = niters
        
        self.test_block = P[k * (len(P) / (k + 1)) : (k + 1) * (len(P) / (k + 1))]
        
        blocks = list()
        for i in range(k):
            block = P[i * (len(P) / (k + 1)) : (i + 1) * (len(P) / (k + 1))]
            blocks.append(block)
        self.blocks = blocks

    def cross_validation(self, lr, reg):
        """ Given the permutation P in the class, evaluate the SVM using k-fold cross validation for the given parameters 
            over the permutation
            Args: 
                lr (float) : learning rate
                reg (float) : regularizer parameter
            Output: 
                (float) : the cross validated error rate
        """
        avg_perf = 0.0

        X_cv = [self.X[self.blocks[i]] for i in range(self.k)]
        y_cv = [self.y[self.blocks[i]] for i in range(self.k)]
        
        for j in range(self.k):

            X_train = sp.vstack(X_cv[ : j] + X_cv[j+1 : ])
            y_train = np.hstack(y_cv[ : j] + y_cv[j+1 : ])

            X_validate = X_cv[j]
            y_validate = y_cv[j]

            mod = SVM(X_train, y_train, reg)
            mod.train(self.niters, lr)

            y_pred = mod.predict(X_validate)
            
            accy = float(np.sum(y_pred != y_validate)) / float(X_validate.shape[0])
            avg_perf = avg_perf + accy
        
        avg_perf = float(avg_perf) / float(self.k)
        return avg_perf
    
    def grid_search(self, lrs, regs):
        """ Given two lists of parameters for learning rate and regularization parameter, perform a grid search using
            k-wise cross validation to select the best parameters. 
            Args:  
                lrs (list) : list of potential learning rates
                regs (list) : list of potential regularizers
            Output: 
                (lr, reg) : 2-tuple of the best found parameters
        """
        opt_params = (lrs[0], regs[0])
        for i in range(len(lrs)):
            for j in range(len(regs)):
                (opt_lr, opt_reg) = opt_params
                if self.cross_validation(lrs[i], regs[j]) < self.cross_validation(opt_lr, opt_reg):
                    opt_params = (lrs[i], regs[j])
        return opt_params
    
    def test(self, lr, reg):
        """ Given parameters, calculate the error rate of the test data given the rest of the data. 
            Args: 
                lr (float) : learning rate
                reg (float) : regularizer parameter
            Output: 
                (err, svm) : tuple of the error rate of the SVM on the test data and the learned model
        """
        X_training0 = [self.X[self.blocks[i]] for i in range(self.k)]
        y_training0 = [self.y[self.blocks[i]] for i in range(self.k)]
        X_test = self.X[self.test_block]
        y_test = self.y[self.test_block]
        
        X_training = sp.vstack(X_training0[ : 3] + X_training0[3 : ])
        y_training = np.hstack(y_training0[ : 3] + y_training0[3 : ])
        
        svm = SVM(X_training, y_training, reg)
        svm.train(self.niters, lr)
        y_pred = svm.predict(X_test)
        err = float(np.sum(y_pred != y_test)) / float(y_pred.shape[0])
        
        return (err, svm)

## K-fold cross validation
How can we evaluate our choice of parameters? One way is to perform k-fold cross validation, which operates as follows 

1. We split the data into k+1 randomly selected but uniformly sized pieces, and set aside one block for testing
2. For each of the remaining k parts, we train the model on k-1 parts and validate our model on the heldout part. 
3. This gives k results, and the average of these runs gives the final result

The idea is that by holding out part of the dataset as validation data, we can train and measure our generalization ability. Note the key fact here: the training does not see the validation data at all, which is why it measures generalization! Randomizing the groups removes bias from ordering (i.e. if these results occurred in chronological order, we don't want to train on only Monday's results to predict on Wednesday's results), and averaging over the groups reduces the variance. 

In this problem, we will use classification error rate as our result metric (so the fraction of times in which our model returns the wrong answer). Calculating this value via k-fold cross validation gives us a measure of how well our model generalizes to new data (lower error rate is better). 

### Specification
1. Break the examples in k+1 groups as follows: 
    * break the permutation into blocks of size $\text{ceil}\left(\frac{n}{k+1}\right)$ (the last block may be shorter than the rest)
    * set aside the k+1th group as the testing block, and use the remaining k blocks for cross validation
    * use the permutation as indices to select the rows that correspond to that block
    * Example: k=2, P=[1,3,2,4,5,6] sets aside [5,6] as the test set, and break the remaining permutation into [[1,3],[2,4]] so the blocks of data for validation are X[[1,3],:] and X[[2,4],:]
    * the order of the indices in the blocks should match the order in the original permutation
2. For each group k, train the model on all other datapoints, and compute the error rate on the held-out group. 
3. Return the average error rate over all k folds, along 

You can try it on the random dataset just to make sure it works, but you won't get anything meaningful. 

In [ ]:
# AUTOLAB_IGNORE_START
MS0 = ModelSelector(X0, y0, np.arange(X0.shape[0]), 3, 100)
MS0.cross_validation(0.1, 1e-4)

# AUTOLAB_IGNORE_STOP

Try running this on the tfidf features. Can you achieve the same performance on the validation dataset as you did on the training data set? Remember to use a random permutation (you'll get noticeably different results). 

In [ ]:
# AUTOLAB_IGNORE_START
# MS0 = ...
# MS0.cross_validation(...)

# AUTOLAB_IGNORE_STOP

Our implementation returns results with mean classification error 0.01169 and standard deviation 0.0092 (over 10 different permutations). The parameters we used are k=5 folds for learning rate 1 and regularization 1e-4, when run for 100 iterations. Pretty good!

## Grid search
Now, we have a means of evaluating our choice of parameters. We can now combine this with a grid search over parameters to determine the best combination. Given two lists of parameters, we compute the classification error using k-fold cross validation for each pair of parameters, and output the parameters that produces the best validation result. 

### Specification
1. Select the pair of hyperparamers that produces the smallest k-fold validation error. 
2. Train a new model using all the training and validation data
3. Report the classification accuracy on the test data

In [ ]:
# MS = ModelSelector(...)
# lr, reg = MS.grid_search(...)
# print lr, reg
# print MS.test(lr,reg)

# AUTOLAB_IGNORE_START
MS = ModelSelector(features, labels, np.arange(features.shape[0]), 4, 100)
lr, reg = MS.grid_search(np.logspace(-1,1,3), np.logspace(-2,1,4))
print lr, reg
print MS.test(lr,reg)
# AUTOLAB IGNORE_STOP

Again, you can try it on the randomized small example just to make sure your code runs, however it won't produce any sort of meaningful result. On our implementation, performing a grid search on learning rates [0.1, 1, 10] and regularization [0.01, 0.1, 1, 10] with 100 iterations for training results in a final test error rate of 0.0232 and selects a learning rate of 1, and a regularization parameter of 0.1. Our implementation takes about 1 minute and 7 seconds to perform the grid search. 

## Feature Compression
While you are able to get decent results using an SVM and basic tf-idf features, there are 2 main problems here:
1. The actual dataset is 8x larger than the one that you load at the start
2. The number of features is extremely bloated and consumes a lot of space and computing power for a binary classification problem

So the above methodology would actually take a lot of time and memory to run on the full dataset. Following the example you did in the text classification notebook, we would need to save the tf-idf matrix for the entire training dataset (which is enormous), and then use that to generate features on new examples. 

One way to tackle this is to generate fewer, but effective, features. For example, instead of generating full tf_idf features for every single word in every email, we can instead try to focus on keywords that frequently only occur in spam email.

In [ ]:
def find_frequent_indicator_words(docs, y, threshold):
    spam_counts = Counter(w for (i, d) in enumerate(docs) for w in d.split() if y[i] == -1)
    ham_counts = Counter(w for (i, d) in enumerate(docs) for w in d.split() if y[i] == 1)
    spam_words = set(w for (i, d) in enumerate(docs) for w in d.split() if y[i] == -1)
    ham_words = set(w for (i, d) in enumerate(docs) for w in d.split() if y[i] == 1)
    
    spam_dict = [w for w in (spam_words - ham_words) if spam_counts[w] >= threshold]
    ham_dict = [w for w in (ham_words - spam_words) if ham_counts[w] >= threshold]
    return spam_dict, ham_dict

In [ ]:
# AUTOLAB_IGNORE_START
s,h = find_frequent_indicator_words(emails, labels, 50)
with open('student_data.pkl', 'wb') as f:
    pickle.dump((s,h), f)
# AUTOLAB_IGNORE_STOP

Our implementation gets 2422 spam words and 290 ham words. 

## Efficient Spam Detection

Your goal here is to get at least 80% accuracy on spam detection in an efficient manner. If you are unsure of what to do, one way is to use the frequent indicator words implemented above and generate 2 features per emails: the number of spam indicator words and the number of ham indicator words for a total of two features. This is a huge dimensionality reduction!

Of course, you don't have to do this. As long as you achieve at least 80% accuracy with your features you will receive the base credit for this problem. You are allowed to submit supplemental files. Make sure these supplemental files make it into your tar file (update the Makefile if you use it).

In [ ]:
def email2features(emails):
    """ Given a list of emails, create a matrix containing features for each email. """
    with open('student_data.pkl', 'rb') as f:
        data = pickle.load(f)
    features = np.zeros((len(emails), 2))
    for i, e in enumerate(emails):
        count = Counter(w for w in e.split())
        features[i, 0] = sum(count[s0] for s0 in data[0])
        features[i, 1] = sum(count[s0] for s0 in data[1])
    return features

In [ ]:
# AUTOLAB_IGNORE_START
small_features = email2features(emails)
# MS = ModelSelector(...)
# lr, reg = MS.grid_search(...)
# print lr, reg
# err, svm = MS.test(lr,reg)
# print err
# AUTOLAB_IGNORE_STOP

## Author: Hexing Ren
### Click [here](http://www.hexingren.com/practical-data-science) to go back.